In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import pandas as pd
from transformers import pipeline
from flask import Flask, render_template, request

c:\Users\HP\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#used for sentence embeddings for vector DB storage
encoder = SentenceTransformer('all-MiniLM-L6-v2') 

## Setting up vector database

In [3]:
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [4]:
# Create collection to store books
qdrant.recreate_collection(
    collection_name="thesis-info",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), 
        distance=models.Distance.COSINE
    )
)

C:\Users\HP\AppData\Local\Temp\ipykernel_17044\2414972111.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

## Reading and storing data

In [5]:
file_path = 'dataset.xlsx'
data_list = []

In [6]:
with pd.ExcelFile(file_path) as xls:
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name, skiprows=1)
            for index, row in df.iterrows():
                documents = {
                    'title': ''.join([word for word in row[1].split('\n')[1:] if word]),
                    'abstract': row[3]
                }
                data_list.append(documents)

C:\Users\HP\AppData\Local\Temp\ipykernel_17044\1050692152.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'title': ''.join([word for word in row[1].split('\n')[1:] if word]),
C:\Users\HP\AppData\Local\Temp\ipykernel_17044\1050692152.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'abstract': row[3]
C:\Users\HP\AppData\Local\Temp\ipykernel_17044\1050692152.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'title': ''.join([

In [7]:
qdrant.upload_records(
    collection_name="thesis-info",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc["abstract"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data_list)
    ]
)

C:\Users\HP\AppData\Local\Temp\ipykernel_17044\1671314003.py:1: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qdrant.upload_records(


In [9]:
hits = qdrant.search(
    collection_name="thesis-info",
    query_vector=encoder.encode("language").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'title': 'Muhammad Umair Arshad (MS-CS)Supervisor: Dr. Waseem Shahzad', 'abstract': 'Typically, there are words in all languages which may have multiple senses/meanings. The meaning/sense associated with a word is referred to as the Word Sense. In ad-dition, ascertaining the correct sense of such a word with reference to its context and use is called Word Sense Disambiguation (WSD). This research focuses on WSD for the Urdu language. Previously, a lot of work on WSD has been performed for the English language, but WSD for Urdu is still in infancy due to rich morphological structure of the language. Therefore, there is a need to explore whether the techniques used for other languages work well on the Urdu language. Moreover, a number of initiatives have resulted in the development of the WSD corpora benchmark for a wide range of languages from different language families. Nevertheless, there is an absence of bench-mark WSD corpora for South Asian languages including Urdu, despite more 

## Integrating LLM and testing prompt generation

In [18]:
generator = pipeline("text-generation", model="gpt2")

def query_vector_database(user_input):
    hits = qdrant.search(
    collection_name="thesis-info",
    query_vector=encoder.encode(user_input).tolist(),
    limit=1
)
    for hit in hits:
      return hit.payload

def generate_response(user_input, relevant_theses):
    prompt = f"User input: {user_input}\n Relevant theses: {relevant_theses}\n"
    generated_text = generator(prompt, max_length=500, num_return_sequences=1, truncation=True)[0]['generated_text']
    return generated_text.strip()

user_input = "what do you know about Prosodic alignment for Automatic dubbing"

relevant_theses = query_vector_database(user_input)
response = generate_response(user_input, relevant_theses)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [19]:
print(response)

User input: what do you know about Prosodic alignment for Automatic dubbing
 Relevant theses: {'title': 'Prosodic alignment for Automatic dubbing', 'abstract': 'Automatic dubbing is the process of replacing the audio track of a video with a different language. In automatic dubbing, prosodic alignment is used to match the suprasegmental features like timbre, prosody, duration, pauses and intonation of the original speech with synthesed speech, in order to produce a natural-sounding dubbed video. This is done by analyzing and mapping these features of the original and translated speech. Existing research on automatic dubbing lack to addresses these features in source video which impact the overall naturalness and fluency of Synthesized speech. To solve this we proposed end-to-end architecture, following modular approach, to generate high quality dubbed video. In this research, we mainly focus on TTS module by considering mentioned features for prosodic alignment in generated speech with 